In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from helper import *

%reload_ext autoreload
%autoreload 2

# Model 1


In [ ]:
water_depth = 70
ocean_coverage = 0.7
shell_volume = surface_area(radius=R_E) * 70
water_volume = ocean_coverage * shell_volume

rho_water_grams = 1.025e6
water_mass_grams = rho_water_grams * water_volume

# Total heat capacity
specific_heat_capacity_water = 4.186
C = water_mass_grams * 4.186

# Heat capacity per unit surface area
C_prime = rho_water_grams * water_depth * ocean_coverage * specific_heat_capacity_water

In [ ]:
S_0 = 1370
alpha = 0.32

E_gain = energy_flux_gain(alpha, S_0 / 4) * surface_area(R_E)
emissivity = 0.65  # 0.65 - 0.99


def f(t, y):
    flux_loss = emissivity * sigma * y[0] ** 4
    E_loss = flux_loss * surface_area(R_E)
    return (E_gain - E_loss) / C


t_f = 20 * 365 * 24 * 60 * 60
t_eval = np.linspace(0, t_f, 1000)
sol = solve_ivp(f, [0, t_f], [255], t_eval=t_eval)

In [ ]:
plt.plot(sol.t, sol.y[0])

# Albedo


In [ ]:
temperatures_0 = np.ones_like(area_fraction) * 203
solar_multiplier = 1.0
time_interval_years = 100.0


def solve(temperatures_0, solar_multiplier):

    def f(t, y):
        temperatures = y
        flux_loss = energy_flux_loss(
            temperatures, average_temperature(area_fraction, temperatures)
        )
        flux_gain = energy_flux_gain(
            albedo(temperatures), local_solar_flux_along_normal, solar_multiplier
        )
        flux_net_gain = flux_gain - flux_loss
        return flux_net_gain / C_prime

    t_f = time_interval_years * 365 * 24 * 60 * 60
    t_eval = np.linspace(0, t_f, 1000)
    sol = solve_ivp(f, [0, t_f], temperatures_0, t_eval=t_eval)

    return sol.y[:, -1]

In [ ]:
def solve_all(multipliers):
    final_temperatures = []
    albedos = []
    initial_temperatures = np.ones_like(area_fraction) * 203

    for M in multipliers:
        initial_temperatures = solve(initial_temperatures, M)
        # print(initial_temperatures)
        final_temperature = np.mean(initial_temperatures)
        final_temperatures.append(final_temperature)
        albedos.append(albedo(final_temperature))
    return final_temperatures, albedos

In [ ]:
multipliers_decr = np.linspace(1.4, 0.6, 100)
multipliers_incr = np.linspace(0.6, 1.4, 100)

temperatures_decr, albedos_decr = solve_all(multipliers_decr)
temperatures_incr, albedos_incr = solve_all(multipliers_incr)

In [ ]:
plt.plot(multipliers_decr, temperatures_decr, label="decreasing")
plt.plot(multipliers_incr, temperatures_incr, linestyle="--", label="increasing")
plt.legend()

In [ ]:
plt.plot(multipliers_decr, albedos_decr, label="decreasing")
plt.plot(multipliers_incr, albedos_incr, linestyle="--", label="increasing")
plt.legend()

In [ ]:
multipliers = np.linspace(0.6, 1.4, 50)
initial_temperatures = np.linspace(203, 320, 50)

mv, tv = np.meshgrid(multipliers, initial_temperatures)

tf_v = np.ones_like(mv)

for i in range(len(mv)):
    print(i)
    for j in range(len(mv[0])):
        tf_v[i][j] = np.mean(
            solve(
                temperatures_0=np.ones_like(area_fraction) * tv[i][j],
                solar_multiplier=mv[i][j],
            )
        )

In [ ]:
plt.contourf(multipliers, initial_temperatures, tf_v - tv)
plt.colorbar()

v, u = np.gradient(tf_v - tv)
n_sample = 2
size = (50 // n_sample) ** 2
u_vec = u[::n_sample, ::n_sample].reshape(size)
v_vec = v[::n_sample, ::n_sample].reshape(size)
vecs = np.array([u_vec, v_vec]).T
vecs_normed = vecs / np.linalg.norm(vecs, axis=1, keepdims=True)
plt.quiver(
    mv[::n_sample, ::n_sample].reshape(size),
    tv[::n_sample, ::n_sample].reshape(size),
    vecs_normed[:, 0],
    vecs_normed[:, 1],
)